<a href="https://colab.research.google.com/github/andsonandreribeiro09/Quantum_Risk_Analysis/blob/main/Quantum_Risk_Analysis_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 Quantum Risk Analysis: Uma Aplicação em Finanças
Baseado no artigo: *Quantum Risk Analysis* (Woerner & Egger, 2019)

Este notebook demonstra como a computação quântica pode ser aplicada à análise de risco financeiro usando VaR e CVaR.

In [1]:
# Instalação das bibliotecas necessárias
!pip install qiskit qiskit-aqua matplotlib numpy pandas --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.9/645.9 kB 15.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.2/368.2 kB 29.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install backend dependencies did not run successfully.


In [5]:
!pip install qiskit

  Using cached qiskit-2.1.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached stevedore-5.4.1-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 10.6 MB/s eta 0:00:00


In [12]:
!pip install qiskit providers aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 3.1.1 requires click>=8.1.3, but you have click 7.1.2 which is incompatible.
dask-cuda 25.2.0 requires click>=8.1, but you have click 7.1.2 which is incompatible.
dask 2024.12.1 requires click>=8.1, but you have click 7.1.2 which is incompatible.
distributed 2024.12.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.
typer 0.16.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.


In [15]:
!pip install qiskit-aer --quiet


In [ ]:
# Imports
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_finance.applications.estimation import CVaRMeasurement
from qiskit_finance.circuit.library import LogNormalDistribution
from qiskit.algorithms import AmplitudeEstimation
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Simulando uma distribuição log-normal
num_qubits = 3
mu = 1.5
sigma = 1.0
distribution = LogNormalDistribution(num_qubits, mu=mu, sigma=sigma)
distribution.draw()

In [ ]:
# Definindo a função de payoff
strike_price = 2.0
low = distribution.low[0]
high = distribution.high[0]
breakpoints = np.linspace(low, high, 2**num_qubits)

def payoff(x):
    return np.maximum(0, x - strike_price)

values = payoff(breakpoints)
normalized = (values - values.min()) / (values.max() - values.min())

In [ ]:
# Medida CVaR
alpha = 0.95
cvar_measurement = CVaRMeasurement(values=normalized, alpha=alpha)
ae = AmplitudeEstimation(num_eval_qubits=3, quantum_instance=Aer.get_backend('qasm_simulator'))
problem = cvar_measurement.construct_estimation_problem(distribution)
result = ae.estimate(problem)
print("Estimativa de CVaR (normalizado):", result.estimation)

In [ ]:
# Comparação com Monte Carlo
from scipy.stats import lognorm
samples = lognorm(s=sigma, scale=np.exp(mu)).rvs(100000)
payoff_mc = payoff(samples)
var_classico = np.quantile(payoff_mc, 1 - alpha)
cvar_classico = payoff_mc[payoff_mc >= var_classico].mean()

print("CVaR Monte Carlo:", cvar_classico)

In [ ]:
# Visualização dos resultados
plt.hist(payoff_mc, bins=50, alpha=0.6, label='Monte Carlo')
plt.axvline(x=cvar_classico, color='r', linestyle='--', label='CVaR Clássico')
plt.axvline(x=result.estimation[0]*values.max(), color='g', linestyle='--', label='CVaR Quântico')
plt.title("Comparação CVaR - Monte Carlo vs Quantum AE")
plt.xlabel("Payoff")
plt.ylabel("Frequência")
plt.legend()
plt.show()

## Conclusão

Este notebook demonstrou o uso da computação quântica para análise de risco financeiro. Usando Quantum Amplitude Estimation (QAE), conseguimos estimar métricas como CVaR com convergência mais rápida que Monte Carlo clássico.

Este é um exemplo promissor de como algoritmos quânticos podem impactar as finanças quantitativas no futuro próximo.